Examining why calculations fail for the triangular graphitic cluster we used when first attempting oniom calculcations to [graphene diels alder reactions](Graphene_oniom_cycloadditions.ipynb)

In [1]:
#import pandas, os
import numpy as np



from webbrowser import open as open_log
from IPython.core.display import HTML, Image
from ase.io import read, write
from gausspy import Gaussian
from glob import glob
from ase_extensions.ase_utils import *
from cc_notebook.cc_notebook_utils import *
#pandas.set_printoptions(max_rows=200, max_columns=10)
#ad_disp.extended_styles()

/home/clyde/anaconda/envs/biotools/lib/python2.7/site-packages/setuptools-18.1-py2.7.egg/pkg_resources/__init__.py:1256: UserWarning: /home/clyde/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).


<IPython.core.display.Javascript object>

In [2]:
cd triang_graph/

/home/clyde/Dropbox/Project Stuff/Notebooks/triang_graph


In [3]:
tri_graph_init, tri_graph_small_init = 'triang_graphene.xyz', 'triang_graphene_small.xyz'
view_ipython_jmol([tri_graph_small_init,tri_graph_init], width=550, height=550)

The small graphene unit has 13 carbons hence an odd number of p electrons so we will focus on the larger fragment (which was the one causing us problems in the first place). As we expect this to fail we will also try using the scf='xqc' keyword:

In [4]:
from IPython.display import display, Javascript
display(Javascript('''
require(['nbextensions/cc_notebook'])
require(['nbextensions/jsmol/JSmol.GLmol.min.js'])
'''))

<IPython.core.display.Javascript object>

Try the normal a single point calculation at the geometry given by avogadro's forcefield optimisation:

In [6]:
#to rerun calcs:

#gaus_objs = []
#stable, mult =['', 'opt'] , [1,3,5]

#for stab in stable:
#    calc_label = 'B3LYP_triang_graphene_1' if not stab else 'B3LYP_triang_graphene_1_stable'
#    gaus_objs.append(Gaussian(label=calc_label, method=B3LYP, basis='6-31G(d)', scf='xqc', multiplicity=1, stable=stab, nosymmetry=''))

#for sp in mult:
#    for stab in stable:
#        calc_label = 'UB3LYP_triang_graphene_{mu}'.format(mu=sp) if not stab else 'UB3LYP_triang_graphene_{mu}_stable'.format(mu=sp)
#        gaus_objs.append(Gaussian(label=calc_label, method=UB3LYP, basis='6-31G(d)', multiplicity=sp, stable=stab, nosymmetry=''))
    
#for stab in stable:
#    calc_label = 'UB3LYP_triang_graphene_{mu}_nosymm'.format(mu=sp) if not stab else 'UB3LYP_triang_graphene_{mu}_stable'.format(mu=sp)
#    gaus_objs.append(Gaussian(label=calc_label, method=UB3LYP, basis='6-31G(d)', multiplicity=3p, stable=stab, nosymmetry='nosymmetry')))
    
#calcs = []
#for gaus_calc in gaus_objs
#    tri=read(tri_graph_init)
#    tri.set_calculator(gaus_calc)
#    tri.calc.set_job(nodes=8, memory=8*1600, time=5, queue='pqmb')
#    calcs.append(tri)

In [7]:
b3lyp_tri = read(tri_graph_init)
b3lyp_tri.set_calculator(Gaussian(label='triang_graphene', method='B3LYP', basis='6-31G(d)'))
b3lyp_tri.calc.set_job(nodes=16, memory=16*2000, time=5, queue='pqmb')
#b3lyp_tri.calc.start()

In [7]:
b3lyp_tri_xqc = read(tri_graph_init)
b3lyp_tri_xqc.set_calculator(Gaussian(label='triang_graphene_xqc', method='B3LYP', basis='6-31G(d)', scf='xqc'))
b3lyp_tri_xqc.calc.set_job(nodes=16, memory=16*2000, time=5, queue='pqmb')
b3lyp_tri_xqc.calc.description = 'RB3LYP'
#b3lyp_tri_xqc.get_potential_energy()

In [8]:
view_ipython_jmol(b3lyp_tri)

In [8]:
check_calcs([b3lyp_tri, b3lyp_tri_xqc])
mols_to_html([b3lyp_tri, b3lyp_tri_xqc])

triang_graphene,,Error: Convergence failure -- run terminated.
triang_graphene_xqc,,Success


In [ ]:
b3lyp_tri.calc.base_folder

As before the xqc option allows the scf to converge whereas it fails without it. But is it stable?

In [8]:
b3lyp_tri_stable = copy.deepcopy(b3lyp_tri_xqc)
b3lyp_tri_stable.calc.restart(add_label='stable', stable='opt')
b3lyp_tri_stable.calc.description = 'RB3LYP, stable=opt'
#b3lyp_tri_xqc.calc.start()

In [9]:
#check_calcs([b3lyp_tri_stable])
mols_to_html([b3lyp_tri_stable])

triang_graphene_xqc_stable_restart_1,com,,Success


<font size=3 color=red>RHF -> UHF instability</font>

In [10]:
print(b3lyp_tri_stable.calc.max_data.keys())

['scfenergies', 'moenergies', 'Rmsd', 'Title', 'S2', 'atomnos', 'atomcoords', 'charge', 'Dipole', 'homos', 'Person', 'Charge', 'Version', 'Pg', 'nmo', 'mosyms', 'natom', 'Root', 'Method', 'mult', 'Chemical_Formula', 'scftargets', 'Multiplicity', 'S2A', 'S2-1', 'atomcharges', 'nbasis', 'State', 'Sequence Number', 'scfvalues', 'Atomic_Numbers', 'coreelectrons', 'atommasses', 'Quadrupole', 'Positions', 'Date', 'Basis_Set', 'Hf', 'Computer_System', 'Type_Of_Run']


In [11]:
b3lyp_tri_stable.calc.max_data['S2A'], b3lyp_tri_stable.calc.max_data['Hf'] - b3lyp_tri_xqc.calc.max_data['Hf']

(0.317148, -0.026818199999979697)

But 22 carbons therefore 22 p electrons = 4n+2 so we would have expected paired electrons? No [Huckel's rule](http://en.wikipedia.org/wiki/H%C3%BCckel's_rule) only applies to single ring systems in fact the symmetric geometry is at a conical intersection see: [Mike's paper on this system](http://www.sciencedirect.com/science/article/pii/S1010603007002390)

Let's try unrestricted calculations:

In [12]:
b3lyp_tri_v2 = read(tri_graph_init)
b3lyp_tri_v2.set_calculator(Gaussian(label='triang_graphene_v2', method='UB3LYP', basis='6-31G(d)'))
b3lyp_tri_v2.calc.set_job(nodes=16, memory=16*2000, time=5)
b3lyp_tri_v2.calc.description = 'UB3LYP, mult=1'
#b3lyp_tri_v2.get_potential_energy()

In [13]:
b3lyp_tri_xqc_v2 = read(tri_graph_init)
b3lyp_tri_xqc_v2.set_calculator(Gaussian(label='triang_graphene_xqc_v2', method='UB3LYP', basis='6-31G(d)', scf='xqc'))
b3lyp_tri_xqc_v2.calc.set_job(nodes=16, memory=16*2000, time=5)
#b3lyp_tri_xqc_v2.get_potential_energy()

In [14]:
check_calcs([b3lyp_tri_v2, b3lyp_tri_xqc_v2])
mols_to_html([b3lyp_tri_v2, b3lyp_tri_xqc_v2])

triang_graphene_v2,com,,Success
triang_graphene_xqc_v2,com,,Success


In [15]:
b3lyp_tri_v2.calc.max_data['S2A'], b3lyp_tri_v2.calc.energy_zero - b3lyp_tri_xqc.calc.energy_zero

(0.288534, -0.6896456114918692)

Is this stable?

In [16]:
b3lyp_tri_v2_stable = copy.deepcopy(b3lyp_tri_v2)
b3lyp_tri_v2_stable.calc.restart(stable='opt')
b3lyp_tri_v2_stable.calc.set_job(nodes=8, memory=8*1600, time=5)
b3lyp_tri_v2_stable.calc.description = 'UB3LYP, mult=1, stable=opt'
#b3lyp_tri_v2_stable.calc.start()
check_calcs([b3lyp_tri_v2_stable])
mols_to_html([b3lyp_tri_v2_stable])

triang_graphene_v2_restart_1,com,,Success


No there was an 'internal instability'

We've been setting multiplicity to 1 but in this case the stability calculations reveal some net spin. Let's try running the calculation with mult=3:

In [17]:
b3lyp_tri_v3 = read(tri_graph_init)
b3lyp_tri_v3.set_calculator(Gaussian(label='triang_graphene_v3', method='UB3LYP', basis='6-31G(d)', multiplicity=3))
b3lyp_tri_v3.calc.set_job(nodes=12, memory=12*1600, time=5)
b3lyp_tri_v3.calc.description = 'UB3LYP, mult=3'
#b3lyp_tri_v3.calc.start()

In [18]:
check_calcs([b3lyp_tri_v3])
mols_to_html([b3lyp_tri_v3])

triang_graphene_v3,com,,Success


In [19]:
b3lyp_tri_v3.calc.max_data['S2A'], b3lyp_tri_v3.calc.energy_zero - b3lyp_tri_xqc.calc.energy_zero

(2.00843, -1.032277899888868)

Is this stable?

In [20]:
b3lyp_tri_v3_stable = copy.deepcopy(b3lyp_tri_v3)
b3lyp_tri_v3_stable.calc.restart(stable='opt')
b3lyp_tri_v3_stable.calc.description = 'UB3LYP, mult=3, stable=opt'
#b3lyp_tri_v3.calc.start()
check_calcs([b3lyp_tri_v3_stable])
mols_to_html([b3lyp_tri_v3_stable])

triang_graphene_v3_restart_1,com,,Success


Yes wavefunction is stable.

In [21]:
b3lyp_tri_v3.calc.max_data['S2A'], b3lyp_tri_v3.calc.energy_zero - b3lyp_tri_xqc.calc.energy_zero

(2.00843, -1.032277899888868)

Just to be sure let's completely remove symmetry:

In [22]:
b3lyp_tri_v4 = read(tri_graph_init)
b3lyp_tri_v4.set_calculator(Gaussian(label='triang_graphene_v4', method='UB3LYP', basis='6-31G(d)', multiplicity=3, nosymmetry='nosymmetry'))
b3lyp_tri_v4.calc.set_job(nodes=12, memory=12*1600, time=5)
b3lyp_tri_v4.calc.description = 'UB3LYP, mult=3, nosymm'
#b3lyp_tri_v4.calc.start()

In [23]:
check_calcs([b3lyp_tri_v4])
mols_to_html([b3lyp_tri_v4])

triang_graphene_v4,com,,Success


In [24]:
b3lyp_tri_v4_stable = copy.deepcopy(b3lyp_tri_v4)
b3lyp_tri_v4_stable.calc.restart(stable='opt')
b3lyp_tri_v4_stable.calc.description = 'UB3LYP, mult=3, nosymm, stable=opt'
#b3lyp_tri_v4_stable.calc.start()

In [25]:
check_calcs([b3lyp_tri_v4_stable])
mols_to_html([b3lyp_tri_v4_stable])

triang_graphene_v4_restart_1,com,,Success


And for completeness let's try an unreasonabl high multiplicity:

In [26]:
b3lyp_tri_v5 = read(tri_graph_init)
b3lyp_tri_v5.set_calculator(Gaussian(label='triang_graphene_v5', method='UB3LYP', basis='6-31G(d)', multiplicity=5, nosymmetry='nosymmetry'))
b3lyp_tri_v5.calc.set_job(nodes=12, memory=12*1600, time=5)
b3lyp_tri_v5.calc.description = 'UB3LYP, mult=5, nosymm'
#b3lyp_tri_v5.calc.start()

In [27]:
check_calcs([b3lyp_tri_v5])
mols_to_html([b3lyp_tri_v5])

triang_graphene_v5,com,,Success


In [28]:
b3lyp_tri_v5_stable = copy.deepcopy(b3lyp_tri_v5)
b3lyp_tri_v5_stable.calc.restart(stable='opt')
b3lyp_tri_v5_stable.calc.description = 'UB3LYP, mult=5, nosymm, stable=opt'
#b3lyp_tri_v5_stable.calc.start()

In [29]:
check_calcs([b3lyp_tri_v5_stable])
mols_to_html([b3lyp_tri_v5_stable])

triang_graphene_v5_restart_1,com,,Success


In [30]:
from collections import OrderedDict as OD
datify = lambda m: OD([['Calc', m.calc.description],['Energy', m.calc.energy_zero], ['Spin', m.calc.max_data.get('S2A')], ['Energy_delta', m.calc.energy_zero - b3lyp_tri_xqc.calc.energy_zero]])
data = map(datify, [b3lyp_tri_xqc, b3lyp_tri_stable, b3lyp_tri_v2, b3lyp_tri_v2_stable, b3lyp_tri_v3, b3lyp_tri_v3_stable, b3lyp_tri_v4, b3lyp_tri_v4_stable, b3lyp_tri_v5, b3lyp_tri_v5_stable])
ad_disp.html_multi_dict(data)

Calc,Energy,Spin,Energy_delta
'RB3LYP',-23008.773526396573,None,0.0
"'RB3LYP, stable=opt'",-23008.773526396573,0.317148,0.0
"'UB3LYP, mult=1'",-23009.463172008065,0.288534,-0.6896456114918692
"'UB3LYP, mult=1, stable=opt'",-23009.50328704754,0.317152,-0.7297606509673642
"'UB3LYP, mult=3'",-23009.805804296462,2.00843,-1.032277899888868
"'UB3LYP, mult=3, stable=opt'",-23009.805804296462,2.00843,-1.032277899888868
"'UB3LYP, mult=3, nosymm'",-23009.804974348895,2.00843,-1.0314479523221962
"'UB3LYP, mult=3, nosymm, stable=opt'",-23009.804974348895,2.00843,-1.0314479523221962
"'UB3LYP, mult=5, nosymm'",-23005.64902142866,6.001369,3.1245049679127987
"'UB3LYP, mult=5, nosymm, stable=opt'",-23005.64902142866,6.001369,3.1245049679127987


In [31]:
mols_to_html([b3lyp_tri_xqc, b3lyp_tri_stable, b3lyp_tri_v2, b3lyp_tri_v2_stable, b3lyp_tri_v3, b3lyp_tri_v3_stable, b3lyp_tri_v4, b3lyp_tri_v4_stable, b3lyp_tri_v5, b3lyp_tri_v5_stable])

triang_graphene_xqc,com,,Success
triang_graphene_xqc_stable_restart_1,com,,Success
triang_graphene_v2,com,,Success
triang_graphene_v2_restart_1,com,,Success
triang_graphene_v3,com,,Success
triang_graphene_v3_restart_1,com,,Success
triang_graphene_v4,com,,Success
triang_graphene_v4_restart_1,com,,Success
triang_graphene_v5,com,,Success
triang_graphene_v5_restart_1,com,,Success


# Notes:

If we run an unrestricted calculation but specify multiplicity=1 why is this different to a restricted calculation - why do we get some net spin (see calc.max_data['S2A'])? 
This is an error associated with unrestricted calcs, S2A attempts to correct for it but doesn't quite get to zero.

Is there a keyword that allows guassian to pick the lowest energy multiplicity? 
No.